<a href="https://colab.research.google.com/github/pmxfa/sp-shapely/blob/main/sp_timegan_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install synthcity tsbootstrap

# Training

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import TimeSeriesDataLoader
from synthcity.utils.serialization import save_to_file, load_from_file

log.add(sink=sys.stderr, level="INFO")

Mounted at /content/drive
[KeOps] Warning : CUDA libraries not found or could not be loaded; Switching to CPU only.


## preprocessing

In [3]:
# Define file path
file_path = "/content/drive/Shareddrives/sp_env/datasets/Weather/weather.csv"

df = pd.read_csv(file_path)
print(df.head())
print(df.info())
print('missing values: \n', df.isnull().sum())

                  date  p (mbar)  T (degC)  Tpot (K)  Tdew (degC)  rh (%)  \
0  2020-01-01 00:10:00   1008.89      0.71    273.18        -1.33    86.1   
1  2020-01-01 00:20:00   1008.76      0.75    273.22        -1.44    85.2   
2  2020-01-01 00:30:00   1008.66      0.73    273.21        -1.48    85.1   
3  2020-01-01 00:40:00   1008.64      0.37    272.86        -1.64    86.3   
4  2020-01-01 00:50:00   1008.61      0.33    272.82        -1.50    87.4   

   VPmax (mbar)  VPact (mbar)  VPdef (mbar)  sh (g/kg)  ...  wv (m/s)  \
0          6.43          5.54          0.89       3.42  ...      1.02   
1          6.45          5.49          0.95       3.39  ...      0.43   
2          6.44          5.48          0.96       3.39  ...      0.61   
3          6.27          5.41          0.86       3.35  ...      1.11   
4          6.26          5.47          0.79       3.38  ...      0.49   

   max. wv (m/s)  wd (deg)  rain (mm)  raining (s)  SWDR (W/m�)  \
0           1.60     224.3     

In [4]:
# Set seed for reproducibility
np.random.seed(42)

# Convert 'date' to datetime and set as index (preserves chronological order)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.sort_index(inplace=True)

# Exclude 'date' from feature selection (it's now the index anyway)
columns = df.columns  # 'date' is no longer in df.columns
selected_features = np.random.choice(columns, size=10, replace=False)

print(f"Selected features: {selected_features}")

# Keep only the selected features
df = df[selected_features]

Selected features: ['p (mbar)' 'PAR (�mol/m�/s)' 'raining (s)' 'T (degC)' 'sh (g/kg)'
 'VPmax (mbar)' 'wv (m/s)' 'Tdew (degC)' 'max. PAR (�mol/m�/s)'
 'SWDR (W/m�)']


In [5]:
# Keep the latest 5000 rows
df_latest = df.tail(5000)

# Train-test split: 70% for training (for TimeGAN), 30% for testing (TSTR)
train_size = int(0.7 * len(df_latest))
df_train = df_latest.iloc[:train_size]
df_test = df_latest.iloc[train_size:]  # you might use this later for TSTR

# Normalize the data using MinMaxScaler
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(df_train)
df_scaled_train = pd.DataFrame(scaled_train, columns=df_train.columns, index=df_train.index)
scaled_test = scaler.transform(df_test)
df_scaled_test = pd.DataFrame(scaled_test, columns=df_test.columns, index=df_test.index)

# Sequence length for time-series data (dataset = per 10 mins, 36 = 6 hrs)
sequence_length = 36

In [ ]:
temporal_data = []
observation_times = []

# Generate sequences from df_scaled_train only
for start in range(len(df_scaled_train) - sequence_length + 1):
    sequence = df_scaled_train.iloc[start:start + sequence_length].reset_index(drop=True)
    temporal_data.append(sequence)
    observation_times.append(list(range(sequence_length)))  # relative time within the window

# Dummy outcome for TimeGAN (can be used in DataLoader)
dummy_outcome = pd.DataFrame(np.zeros(len(temporal_data)), columns=["outcome"])

# --- Create DataLoader for TimeGAN ---
loader = TimeSeriesDataLoader(
    temporal_data=temporal_data,
    observation_times=observation_times,
    static_data=None,
    outcome=dummy_outcome,
)

# Print the loader info
print(f"TimeSeriesDataLoader created with {len(temporal_data)} sequences")

In [6]:
temporal_data_test = []
observation_times_test = []

# Generate sequences from df_scaled_test only
for start in range(len(df_scaled_test) - sequence_length + 1):
    sequence = df_scaled_test.iloc[start:start + sequence_length].reset_index(drop=True)
    temporal_data_test.append(sequence)
    observation_times_test.append(list(range(sequence_length)))  # relative time within the window

# Dummy outcome for TimeGAN (can be used in DataLoader)
dummy_outcome = pd.DataFrame(np.zeros(len(temporal_data_test)), columns=["outcome"])

# Create DataLoader for TimeGAN
loader_test = TimeSeriesDataLoader(
    temporal_data=temporal_data_test,
    observation_times=observation_times_test,
    static_data=None,
    outcome=dummy_outcome,
)

# Print the loader info
print(f"TimeSeriesDataLoader TEST SET created with {len(temporal_data_test)} sequences")

TimeSeriesDataLoader TEST SET created with 1465 sequences


In [ ]:
print(len(df_scaled_train))  # Check the length of the dataframe
print(loader.dataframe())

3500
        seq_id  seq_time_id  seq_temporal_PAR (�mol/m�/s)  \
0            0            0                      0.000000   
1            0            1                      0.000000   
2            0            2                      0.000000   
3            0            3                      0.000000   
4            0            4                      0.000000   
...        ...          ...                           ...   
124735    3464           31                      0.379520   
124736    3464           32                      0.396154   
124737    3464           33                      0.397787   
124738    3464           34                      0.451815   
124739    3464           35                      0.408922   

        seq_temporal_SWDR (W/m�)  seq_temporal_T (degC)  \
0                       0.000000               0.114042   
1                       0.000000               0.127230   
2                       0.000000               0.148177   
3                       0.

In [ ]:
hparams = {
          "mode": "LSTM", # default mode = RNN;
          "dataloader_sampling_strategy": "none" # default = imbalanced_time_censoring
}`
# Load TimeGAN with custom parameters
syn_model = Plugins().get("timegan", **hparams)

[2025-05-07T10:35:02.239810+0000][3576][CRITICAL] module disabled: /usr/local/lib/python3.11/dist-packages/synthcity/plugins/generic/plugin_goggle.py


In [ ]:
# --- Print all parameters of initialized model ---
for attr in dir(syn_model):
    if not attr.startswith("_") and not callable(getattr(syn_model, attr)):
        print(f"{attr}: {getattr(syn_model, attr)}")

batch_size: 64
class_name: TimeGANPlugin
clipping_value: 0
compress_dataset: False
dataloader_sampling_strategy: none
device: cuda
discriminator_batch_norm: False
discriminator_dropout: 0.1
discriminator_loss: None
discriminator_lr: 0.001
discriminator_n_iter: 1
discriminator_n_layers_hidden: 3
discriminator_n_units_hidden: 300
discriminator_nonlin: leaky_relu
discriminator_weight_decay: 0.001
embedding_penalty: 10
encoder: None
encoder_max_clusters: 20
expecting_conditional: False
fitted: False
gamma_penalty: 1
generator_batch_norm: False
generator_dropout: 0.01
generator_loss: None
generator_lr: 0.001
generator_n_layers_hidden: 2
generator_n_units_hidden: 150
generator_nonlin: leaky_relu
generator_nonlin_out_continuous: tanh
generator_nonlin_out_discrete: softmax
generator_residual: True
generator_weight_decay: 0.001
mode: LSTM
module_name: synthcity.plugins.time_series.plugin_timegan
module_relative_path: ../time_series/plugin_timegan.py
moments_penalty: 100
n_iter: 1000
n_iter_prin

## fitting the model

In [ ]:
print(loader.shape)

# Train the model
syn_model.fit(loader)

(124740, 13)


100%|██████████| 1000/1000 [2:53:03<00:00, 10.38s/it]


In [ ]:
#@title optional: generate data from saved model
from synthcity.utils.serialization import save_to_file, load_from_file

# Save model to drive
save_to_file('/content/drive/Shareddrives/sp_env/saved_models/GAN_Weather.pkl', syn_model)

In [ ]:
# --- Generate Synthetic Data ---
n_samples = len(temporal_data)
print("n_samples:", n_samples)
syn_data = syn_model.generate(count=n_samples)
print(syn_data.shape)

n_samples: 3465
(75161, 13)


In [ ]:
# --- Save with automated format ---
import datetime
import os
# Get the current date and time
now = datetime.datetime.now()
timestamp = now.strftime("%m%d%y-%H%M%S")  # MMDDYY-HHMMSS format

# Define the base directory
base_dir = "/content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/weather"  #CHANGE THIS
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Construct the filename
model_name = type(syn_model).__name__.lower() # Get model name dynamically
filename = f"{timestamp}-{model_name}-n_3500.csv"
filepath = os.path.join(base_dir, filename)

# Save the data
df_syn = syn_data.dataframe()
df_syn.to_csv(filepath, index=False)

print(f"Synthetic data saved to: {filepath}")

Synthetic data saved to: /content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/weather/050725-134006-timeganplugin-n_3500.csv


# Evaluation

## Prerequisites

In [ ]:
loader

,seq_id,seq_time_id,seq_temporal_PAR (�mol/m�/s),seq_temporal_SWDR (W/m�),seq_temporal_T (degC),seq_temporal_Tdew (degC),seq_temporal_VPmax (mbar),seq_temporal_max. PAR (�mol/m�/s),seq_temporal_p (mbar),seq_temporal_raining (s),seq_temporal_sh (g/kg),seq_temporal_wv (m/s),seq_out_outcome
0,0,0,0.000000,0.000000,0.114042,0.391513,0.077118,0.000000,0.837838,0.0,0.292105,0.132780,0.0
1,0,1,0.000000,0.000000,0.127230,0.397918,0.087231,0.000000,0.837838,0.0,0.297368,0.055325,0.0
2,0,2,0.000000,0.000000,0.148177,0.403523,0.102402,0.000000,0.840744,0.0,0.302632,0.051176,0.0
3,0,3,0.000000,0.000000,0.150504,0.403523,0.103666,0.000000,0.841325,0.0,0.302632,0.110650,0.0
4,0,4,0.000000,0.000000,0.149728,0.409127,0.103666,0.000000,0.839872,0.0,0.307895,0.087137,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
124735,3464,31,0.379520,0.341668,0.576416,0.741393,0.479140,0.332190,0.856437,0.0,0.655263,0.412172,0.0
124736,3464,32,0.396154,0.359281,0.577192,0.742994,0.479140,0.339379,0.855856,0.0,0.657895,0.420470,0.0
124737,3464,33,0.397787,0.362148,0.577967,0.744596,0.480405,0.349280,0.854693,0.0,0.657895,0.388658,0.0
124738,3464,34,0.451815,0.419480,0.589604,0.751001,0.493047,0.385717,0.854403,0.0,0.665789,0.271093,0.0


In [8]:
syn_data = pd.read_csv('/content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/weather/050725-134006-timeganplugin-n_3500.csv')

In [9]:
# Define selected columns explicitly
selected_columns = ['seq_temporal_PAR (�mol/m�/s)', 'seq_temporal_SWDR (W/m�)', 'seq_temporal_T (degC)',	'seq_temporal_Tdew (degC)',	'seq_temporal_VPmax (mbar)',	'seq_temporal_max. PAR (�mol/m�/s)',	'seq_temporal_p (mbar)',	'seq_temporal_raining (s)',	'seq_temporal_sh (g/kg)',	'seq_temporal_wv (m/s)']
# Ensure real_data and synthetic_data only contain the selected columns
real_data = loader_test.dataframe()[selected_columns].to_numpy()
synthetic_data = syn_data[selected_columns].to_numpy()

In [10]:
print(real_data, "\n ------------------------------------------------------- \n", synthetic_data)
print(type(real_data),type(synthetic_data))
print(real_data.shape,synthetic_data.shape)

[[0.46769315 0.44037457 0.61830877 ... 0.         0.69210526 0.26002766]
 [0.48242026 0.46101414 0.64701319 ... 0.         0.70789474 0.23651452]
 [0.38726973 0.36198879 0.67416602 ... 0.         0.71315789 0.31258645]
 ...
 [0.         0.         0.28704422 ... 0.         0.28157895 0.03042877]
 [0.         0.         0.29325058 ... 0.         0.27894737 0.05670816]
 [0.         0.         0.28859581 ... 0.         0.28684211 0.05670816]] 
 ------------------------------------------------------- 
 [[0.00000000e+00 0.00000000e+00 6.59008511e-01 ... 0.00000000e+00
  4.20039350e-01 1.35567956e-01]
 [3.76885321e-05 0.00000000e+00 2.42707856e-01 ... 0.00000000e+00
  5.04569243e-01 1.35470773e-01]
 [9.50433284e-06 0.00000000e+00 2.42746615e-01 ... 2.72382253e-06
  5.04461608e-01 1.35184309e-01]
 ...
 [7.69884213e-05 0.00000000e+00 2.92376809e-01 ... 1.47643289e-05
  6.42898767e-01 7.83156616e-02]
 [8.08675309e-05 0.00000000e+00 6.56675326e-01 ... 8.22913082e-06
  6.42783858e-01 2.51136509e-

## Generate distance metrics

### Helper Functions

In [23]:
from scipy.stats import wasserstein_distance, entropy
import numpy as np

def compute_wasserstein(real_data, synthetic_data, selected_columns):
    """
    Computes Wasserstein Distance between real and synthetic time-series data.

    """

    # Ensure both datasets have the same number of samples
    min_length = min(len(real_data), len(synthetic_data))
    real_trimmed = real_data[:min_length]  # Keep original order (no random sampling)
    synthetic_trimmed = synthetic_data[:min_length]  # Match size
    print(real_trimmed.shape,synthetic_trimmed.shape)

    wasserstein_results = {}

    # Compute Wasserstein Distance for each feature
    for i, col in enumerate(selected_columns):
        w_dist = wasserstein_distance(real_trimmed[:, i], synthetic_trimmed[:, i])
        wasserstein_results[col] = w_dist
        print(f"{w_dist}")

    return wasserstein_results

def compute_kl_divergence(real_data, synthetic_data, selected_columns, bins=50):
    """
    Computes KL Divergence between real and synthetic time-series data.

    """

    # Ensure both datasets have the same number of samples
    min_length = min(len(real_data), len(synthetic_data))
    real_trimmed = real_data[:min_length]  # Keep original order
    synthetic_trimmed = synthetic_data[:min_length]  # Match size

    kl_results = {}

    for i, col in enumerate(selected_columns):
        # Compute histogram-based probability distributions
        real_hist, _ = np.histogram(real_trimmed[:, i], bins=bins, density=True)
        synth_hist, _ = np.histogram(synthetic_trimmed[:, i], bins=bins, density=True)

        # Avoid zero probabilities (KL Divergence is undefined for zero values)
        real_hist += 1e-10
        synth_hist += 1e-10

        # Compute KL Divergence
        kl_div = entropy(real_hist, synth_hist)
        kl_results[col] = kl_div
        print(f"{kl_div}")

    return kl_results

### Generate Metrics

In [ ]:
# Assuming df_scaled is the DataFrame containing your scaled ETD data

# Compute Wasserstein Distance
wasserstein_results = compute_wasserstein(real_data, synthetic_data, selected_columns)
print("Wasserstein Distance Results:")
print(wasserstein_results)

# Compute KL Divergence
kl_results = compute_kl_divergence(real_data, synthetic_data, selected_columns)
print("KL Divergence Results:")
print(kl_results)

(52740, 10) (52740, 10)
0.07390652846993843
0.0686342482103673
0.14073753513163795
0.27005066389242843
0.1729592751967025
0.07352453805629322
0.3774205127692291
0.08258920462858084
0.26066581377144693
0.1679205308385537
Wasserstein Distance Results:
{'seq_temporal_PAR (�mol/m�/s)': 0.07390652846993843, 'seq_temporal_SWDR (W/m�)': 0.0686342482103673, 'seq_temporal_T (degC)': 0.14073753513163795, 'seq_temporal_Tdew (degC)': 0.27005066389242843, 'seq_temporal_VPmax (mbar)': 0.1729592751967025, 'seq_temporal_max. PAR (�mol/m�/s)': 0.07352453805629322, 'seq_temporal_p (mbar)': 0.3774205127692291, 'seq_temporal_raining (s)': 0.08258920462858084, 'seq_temporal_sh (g/kg)': 0.26066581377144693, 'seq_temporal_wv (m/s)': 0.1679205308385537}
4.032920919805619
4.24754794057803
10.844459159499756
14.53804597222291
10.94419192007382
4.161905191910566
14.06077938809766
1.3471632239046223
14.678658742901826
10.445804139563247
KL Divergence Results:
{'seq_temporal_PAR (�mol/m�/s)': 4.032920919805619, 's

# LSTM downstream

In [11]:
filepath = '/content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/weather/050725-134006-timeganplugin-n_3500.csv'
real_data = loader_test.dataframe()
df_synth = pd.read_csv(filepath)

# 2. Drop the unwanted column
real_data = real_data.drop(columns=["seq_id", "seq_time_id", "seq_out_outcome"], errors="ignore")
df_synth = df_synth.drop(columns=["seq_id", "seq_time_id", "seq_out_outcome"], errors="ignore")

In [12]:
print(f"real_data: {real_data.shape}, synthetic_data: {df_synth.shape}")

real_data: (52740, 10), synthetic_data: (75161, 10)


In [14]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [15]:
# Convert to tensors (float32 for PyTorch)
data_real = torch.tensor(real_data.values, dtype=torch.float32)
data_synth = torch.tensor(df_synth.values, dtype=torch.float32)

# ──────── Sequence builder ───────────
def make_sequences(data, seq_len):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return torch.stack(X), torch.stack(y)

SEQ_LEN = sequence_length

# Sequences for synthetic (train)
X_train, y_train = make_sequences(data_synth, SEQ_LEN)
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)

# Sequences for real (test)
X_test, y_test = make_sequences(data_real, SEQ_LEN)

In [16]:
# ─── Model Definition ──────────────────────────────────────
class ShallowLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=64):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)  # hn shape: (1, batch, hidden_size)
        out = self.linear(hn.squeeze(0))  # squeeze to (batch, hidden_size)
        return out


# ─── Model Init ─────────────────────────────────────────────
model = ShallowLSTM(input_size=X_train.shape[2], hidden_size=64)

# ─── Optimizer & Loss ───────────────────────────────────────
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

In [17]:

# ─── Training ───────────────────────────────────────────────
EPOCHS = 50
for epoch in range(1, EPOCHS + 1):
    model.train()
    for xb, yb in train_loader:
        pred = model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # if epoch % 10 == 0 or epoch == 1:
    print(f"Epoch {epoch}: Train MSE = {loss.item():.6f}")

Epoch 1: Train MSE = 0.006901
Epoch 2: Train MSE = 0.010589
Epoch 3: Train MSE = 0.008098
Epoch 4: Train MSE = 0.008875
Epoch 5: Train MSE = 0.008013
Epoch 6: Train MSE = 0.010602
Epoch 7: Train MSE = 0.011461
Epoch 8: Train MSE = 0.010880
Epoch 9: Train MSE = 0.011432
Epoch 10: Train MSE = 0.009937
Epoch 11: Train MSE = 0.007814
Epoch 12: Train MSE = 0.011862
Epoch 13: Train MSE = 0.009760
Epoch 14: Train MSE = 0.009160
Epoch 15: Train MSE = 0.015381
Epoch 16: Train MSE = 0.008420
Epoch 17: Train MSE = 0.014596
Epoch 18: Train MSE = 0.008992
Epoch 19: Train MSE = 0.013847
Epoch 20: Train MSE = 0.016404
Epoch 21: Train MSE = 0.010886
Epoch 22: Train MSE = 0.008493
Epoch 23: Train MSE = 0.013442
Epoch 24: Train MSE = 0.007267
Epoch 25: Train MSE = 0.013990
Epoch 26: Train MSE = 0.019234
Epoch 27: Train MSE = 0.011221
Epoch 28: Train MSE = 0.012479
Epoch 29: Train MSE = 0.009458
Epoch 30: Train MSE = 0.011883
Epoch 31: Train MSE = 0.007323
Epoch 32: Train MSE = 0.011511
Epoch 33: Train M

In [18]:
# --- ADD THESE LINES TO SAVE THE MODEL ---
# Define a path where you want to save your model.
# Use a meaningful name, especially for TRTR vs. TSTR models.
# Example for TRTR Electricity model:
MODEL_SAVE_PATH = '/content/drive/Shareddrives/sp_env/saved_models/LSTM/tstr_GAN_weather.pth'

# Save only the model's learned parameters (state_dict)
torch.save(model.state_dict(), MODEL_SAVE_PATH)

print(f"Model saved to: {MODEL_SAVE_PATH}")

Model saved to: /content/drive/Shareddrives/sp_env/saved_models/LSTM/tstr_GAN_weather.pth


In [19]:
model.eval()
with torch.no_grad():
    preds = model(X_test)
    test_mse = loss_fn(preds, y_test).item()
    test_mae = mean_absolute_error(y_test.numpy(), preds.numpy())

    print(f"Test MSE: {test_mse:.6f}")
    print(f"Test MAE: {test_mae:.6f}")

Test MSE: 0.122931
Test MAE: 0.213100


# Bootstrapping

In [26]:
from tsbootstrap import MovingBlockBootstrap
import numpy as np

bootstrap_configs = {
    "weather": {"block_length": 36, "n_bootstraps": 15, "rng": 42},       # 6-hour pattern (10-min interval)
    "electricity": {"block_length": 24, "n_bootstraps": 15, "rng": 42},   # 1-day pattern (hourly)
    "exchange": {"block_length": 30, "n_bootstraps": 15, "rng": 42},      # 1-month pattern (daily)
}

# Example for weather
dataset_name = "weather"
config = bootstrap_configs[dataset_name]

real_test_array = real_data[selected_columns].to_numpy()  # shape (N, features)
mbb = MovingBlockBootstrap(
    n_bootstraps=config["n_bootstraps"],
    rng=config["rng"],
    block_length=config["block_length"]
)
boot_samples = mbb.bootstrap(real_test_array, return_indices=False)


In [27]:
model.load_state_dict(torch.load(MODEL_SAVE_PATH))

<All keys matched successfully>

In [28]:
bootstrap_results = []

for b_idx, boot_real in enumerate(boot_samples):
    # 1. Match the synthetic data size
    syn_trimmed = synthetic_data[:len(boot_real)]

    # 2. Fidelity metrics
    wasserstein = compute_wasserstein(boot_real, syn_trimmed, selected_columns)
    kl = compute_kl_divergence(boot_real, syn_trimmed, selected_columns)

    # 3. Utility metrics
    # Preprocess this bootstrap sample for LSTM (as you do with real_data)
    boot_tensor = torch.tensor(boot_real, dtype=torch.float32)
    Xb_test, yb_test = make_sequences(boot_tensor, SEQ_LEN)

    model.eval()
    with torch.no_grad():
        preds = model(Xb_test)
        mse = mean_squared_error(yb_test.numpy(), preds.numpy())
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(yb_test.numpy(), preds.numpy())

    # 4. Store results
    bootstrap_results.append({
        'bootstrap': b_idx,
        'wasserstein': np.mean(list(wasserstein.values())),
        'kl': np.mean(list(kl.values())),
        'rmse': rmse,
        'mae': mae
    })

(52740, 10) (52740, 10)
0.07296160817783086
0.06780689872180058
0.1438046317662236
0.2756324615483454
0.17622567112761123
0.07171763024974467
0.36737824546614484
0.08044192121438147
0.26842601416493866
0.1604433933739079
4.005361178761365
4.171689909858293
10.92964037780794
14.59346621381363
11.004692529712397
4.129724688502113
14.133321767353376
1.334317212784871
14.782385281005174
10.472618315804388
(52740, 10) (52740, 10)
0.07483086098487894
0.06954366675618179
0.14460368261289863
0.2727353246760528
0.17674439241006604
0.07483955518067162
0.3691845161208136
0.08051367831906288
0.26423082486746147
0.17270454391504178
3.972487647733977
4.196844567513886
10.742232329636362
14.608552999986905
11.168974754536524
4.120909287764697
13.884923396873942
1.3298966154915104
14.590612460252606
10.479118296452041
(52740, 10) (52740, 10)
0.07574950304201007
0.07033742573931687
0.14576761825092635
0.27355093526586216
0.17913543855845712
0.0758977638109247
0.37864959057542896
0.08479935792470311
0.2

In [29]:
print(bootstrap_results)

# Assuming bootstrap_results is your list of dicts
df_results = pd.DataFrame(bootstrap_results)

df_results['Dataset'] = 'Weather'
df_results['Model'] = 'GAN'

df_results

[{'bootstrap': 0, 'wasserstein': 0.1684838475810929, 'kl': 8.955721747540355, 'rmse': 0.343259, 'mae': 0.21043873}, {'bootstrap': 1, 'wasserstein': 0.16999310458431297, 'kl': 8.909455235624245, 'rmse': 0.34387398, 'mae': 0.21209805}, {'bootstrap': 2, 'wasserstein': 0.17231381420550743, 'kl': 8.981011456061278, 'rmse': 0.3488566, 'mae': 0.21456473}, {'bootstrap': 3, 'wasserstein': 0.1709924180344465, 'kl': 8.977745305248634, 'rmse': 0.34822926, 'mae': 0.21355136}, {'bootstrap': 4, 'wasserstein': 0.1678169020765558, 'kl': 8.91185845593186, 'rmse': 0.34376967, 'mae': 0.21052906}, {'bootstrap': 5, 'wasserstein': 0.1672083454574017, 'kl': 8.907097895837573, 'rmse': 0.34336123, 'mae': 0.20995367}, {'bootstrap': 6, 'wasserstein': 0.16744528224275065, 'kl': 8.868644162768941, 'rmse': 0.34566873, 'mae': 0.21000525}, {'bootstrap': 7, 'wasserstein': 0.17030418833987995, 'kl': 8.952479833853465, 'rmse': 0.34681752, 'mae': 0.21290588}, {'bootstrap': 8, 'wasserstein': 0.17265138756670767, 'kl': 8.99

,bootstrap,wasserstein,kl,rmse,mae,Dataset,Model
0,0,0.168484,8.955722,0.343259,0.210439,Weather,GAN
1,1,0.169993,8.909455,0.343874,0.212098,Weather,GAN
2,2,0.172314,8.981011,0.348857,0.214565,Weather,GAN
3,3,0.170992,8.977745,0.348229,0.213551,Weather,GAN
4,4,0.167817,8.911858,0.343770,0.210529,Weather,GAN
5,5,0.167208,8.907098,0.343361,0.209954,Weather,GAN
6,6,0.167445,8.868644,0.345669,0.210005,Weather,GAN
7,7,0.170304,8.952480,0.346818,0.212906,Weather,GAN
8,8,0.172651,8.997477,0.352176,0.216250,Weather,GAN
9,9,0.166412,8.817419,0.343760,0.208554,Weather,GAN


In [30]:
summary_row = {
    'Dataset': 'El',
    'Model': 'GAN',
    'Wasserstein': df_results['wasserstein'].mean(),
    'KL': df_results['kl'].mean(),
    'RMSE': df_results['rmse'].mean(),
    'MAE': df_results['mae'].mean()
}

df_summary = pd.DataFrame([summary_row])
print(df_summary)

  Dataset Model  Wasserstein        KL      RMSE       MAE
0      El   GAN     0.168753  8.908241  0.344892  0.210943
